# CS_BP预处理部分

修改版（改进如下）其中1,4为核心点，已完成；2,3不打算搞了，意义不大。
1：使用反向学习和混沌初始化解决随机初始化种群在整个搜索空间分布不均的问题，整加初始种群的表达能力
2：局部优化的时候随机生成反向的两组，增加局部收敛速度,
3：局部优化修改原始的随机淘汰一部分nest的方式，使用每隔一段周期，计算在在些周期过去之后适应度进化程度百分比最低的nest，因为这些nests很有可能陷入了局部最优。新的位置选择随机两组较优nest之间，保证算法的全局收敛能力。
4:引入adaboost,组合多个CS-BP模型，其中每个CS的nests初始化使用tent初始化，对于tent的初值不再靠实验，而是产生多个足够有差异CS-BP，最后组合优化

In [19]:
import numpy as np
import scipy.special as sc_special
import tensorflow as tf 
from 	tensorflow import keras
import pandas as pd
from tensorflow.keras import optimizers,metrics,layers,Sequential


In [20]:
# 导入数据以及预处理
data = pd.read_csv('DATASETS/GY_AIR.csv',sep=',',header=0,usecols=[1,2,3,4,5,6,7])

data = data.sample(frac = 1) #打乱数据

X = data.iloc[:410,1:]
Y = data.iloc[:410,0]
TestX = data.iloc[410:,1:]
TestY = data.iloc[410:,0]

inputnum = 6
hiddennum = 10
outputnum = 3
numsum = inputnum*hiddennum+hiddennum+hiddennum*outputnum+outputnum #6*10+10+10*3+3=103
bestfit = []  #保存CS迭代过程中的loss、acc，画图用
bestloss = []
LossArr = []


n_train = X.shape[0]
#弱分类器数量
M = 5


def preprocess(x,y):
    x = tf.cast(x,dtype=tf.float32) #/ X.max().max()
    y = tf.cast(y,dtype=tf.int32)
    y = tf.one_hot(y,depth=3)
    return x,y
len(X),len(TestX)

(410, 129)

In [21]:
#数据预处理

#零均值处理
X.iloc[:,0] -= np.mean(X,axis=0)[0]  
X.iloc[:,1] -= np.mean(X,axis=0)[1]
X.iloc[:,2] -= np.mean(X,axis=0)[2]
X.iloc[:,3] -= np.mean(X,axis=0)[3]  
X.iloc[:,4] -= np.mean(X,axis=0)[4]
X.iloc[:,5] -= np.mean(X,axis=0)[5]
#归一化
X.iloc[:,0] /= np.max(np.abs(X),axis=0)[0]
X.iloc[:,1] /= np.max(np.abs(X),axis=0)[1]
X.iloc[:,2] /= np.max(np.abs(X),axis=0)[2]
X.iloc[:,3] /= np.max(np.abs(X),axis=0)[3]  
X.iloc[:,4] /= np.max(np.abs(X),axis=0)[4]
X.iloc[:,5] /= np.max(np.abs(X),axis=0)[5]
#零均值处理
TestX.iloc[:,0] -= np.mean(TestX,axis=0)[0]  
TestX.iloc[:,1] -= np.mean(TestX,axis=0)[1]
TestX.iloc[:,2] -= np.mean(TestX,axis=0)[2]
TestX.iloc[:,3] -= np.mean(TestX,axis=0)[3]  
TestX.iloc[:,4] -= np.mean(TestX,axis=0)[4]
TestX.iloc[:,5] -= np.mean(TestX,axis=0)[5]
#归一化
TestX.iloc[:,0] /= np.max(np.abs(TestX),axis=0)[0]
TestX.iloc[:,1] /= np.max(np.abs(TestX),axis=0)[1]
TestX.iloc[:,2] /= np.max(np.abs(TestX),axis=0)[2]
TestX.iloc[:,3] /= np.max(np.abs(TestX),axis=0)[3]  
TestX.iloc[:,4] /= np.max(np.abs(TestX),axis=0)[4]
TestX.iloc[:,5] /= np.max(np.abs(TestX),axis=0)[5]
batchsz = 16
db = tf.data.Dataset.from_tensor_slices((X,Y))
db = db.map(preprocess).batch(batchsz)#.shuffle(60000)
ds_val = tf.data.Dataset.from_tensor_slices((TestX,TestY))
ds_val = ds_val.map(preprocess).batch(batchsz)
sample = next(iter(db))
print(sample[0].shape,sample[1].shape)
Y_onehot = tf.one_hot(Y,depth=3)
TestY_onehot = tf.one_hot(TestY,depth=3)
X = tf.cast(X,dtype=tf.float32)# / 203.
TestX = tf.cast(TestX,dtype=tf.float32) #/ 203.

(16, 6) (16, 3)


# CS部分

In [22]:

#n：布谷鸟数  m：布谷鸟的维度
def cuckoo_search(n, m, lower_boundary, upper_boundary, iter_num = 20,pa = 0.25, beta = 1.5, step_size = 0.1,alpha=0.77,xn=0.33):

    num=1
    # get initial nests' locations 
    nests,best_nest,best_fitness,lossness = generate_nests(n, m,alpha,xn) #alpha,xn用于tent初始化阈值和初值设置
    
    # get the best nest and record it
    bestfit.append(best_fitness) #全局变量，保存每代最佳适应度值
    

    print('\r\n BEST_LOSSNESS IS %.2f : \r\n',best_fitness)

    for _ in range(iter_num):
        
        print('\r\n******************************************************第 %d 代开始迭代优化************************************************************\r\n'%num)
        nests = update_nests(lower_boundary, upper_boundary, nests, best_nest, lossness, step_size,best_fitness)
        nests = abandon_nests(nests, lower_boundary, upper_boundary, pa)
        
        print('\r\n*****************************************************第 %d 次迭代，计算适应度********************************************************\r\n'%num)
        fitness,lossness,_ = calc_fitness( nests)
        print('\r\n*********************************************************第 %d 次迭代结束************************************************************\r\n'%num)
        
        min_loss_index = np.argmin(lossness)
        min_loss = lossness[min_loss_index]
        min_loss_fit = fitness[min_loss_index]
        min_nestloss = nests[min_loss_index]
        LossArr.append(min_loss)
            
        if min_loss < best_fitness  : #and  min_loss_fit > best_two_fitness
            best_nest = min_nestloss
            best_fitness = min_loss
            best_acc = min_loss_fit
            print('\r\n******')
            print('\r\n 第 %d 次迭代最优Loss是 %.2f : \r\n'%(num,best_fitness))
            print('\r\n******\r\n')
        num+=1

    return (best_nest, best_fitness,best_acc)

In [23]:
def generate_nests(n, m,alpha,xn):

#随机生成nest
#     lower_boundary = np.array(lower_boundary)
#     upper_boundary = np.array(upper_boundary)
#     nests = np.empty((n, m))

#     for each_nest in range(n):
#         nests[each_nest] = lower_boundary + np.array([np.random.rand() for _ in range(m)]) * (upper_boundary - lower_boundary)

#Tent混沌反向初始化

#混沌初始化
    nests = np.empty((n, m))
    sig_nest = np.empty(m)
    alpha = alpha
    xn = xn
    for i in range(0,n):   #*2 值域为【-1,1】 *6 值域为[-3,3]
        for j in range(0,m):
            if 0<=xn<alpha:
                xn = xn/alpha
                sig_nest[j]=(xn-0.5)*6
            elif alpha <= xn <= 1:
                xn = (1-xn)/(1-alpha)
                sig_nest[j] = (xn-0.5)*6
            nests[i] = sig_nest
            
#反向初始化            
    renests = -1 * nests   #定义：Pi = ai + bi - pi  生成反向nests
    
#拼接两个初始化nests
    nests = np.vstack((nests,renests))  #拼接nests和renests 准备计算适应度选择最优的n个nest
    
#计算适应度
    fitness,lossness,_ = calc_fitness( nests) 
    
#根据loss值排序
    arrIndex = np.argsort(lossness)   #获得排序数组  从小到大
    lossness = lossness[arrIndex]     #将lossness数组按照从小到大排序
    nests = nests[arrIndex]    #将nests也按照相同序列进行排序，保证和lossness对齐
    
#删除多余的n组nest，这里从最底下开始一个个删，因为已经排好序了，所以删除的为效果最差的
    for i in range(n):
        nests = np.delete(nests,-1,0)
        lossness = np.delete(lossness,-1,0)
#现在的nests是按照loss排序的，第一个loss最小

    return nests,nests[0],lossness[0],lossness

In [24]:
def update_nests(lower_boundary, upper_boundary, nests, best_nest, lossness, step_coefficient,bestfitness):

    lower_boundary = np.array(lower_boundary)
    upper_boundary = np.array(upper_boundary)
    n, m = nests.shape
    # 莱维飞行生成步长：较长时间的短步长和较短时间的长步长
    #生成的步长的shape也是n*m，保证所有元素得到更新
    #这个步长通过step_size来控制大小
    steps = levy_flight(n, m, 1.5)
    new_nests = nests.copy()

    for each_nest in range(n):
        # coefficient 0.01 is to avoid levy flights becoming too aggresive
        # and (nest[each_nest] - best_nest) 保留了最佳nest，但会导致局部最优
        step_size = step_coefficient * steps[each_nest] # * (nests[each_nest] - best_nest)
        step_direction = np.random.rand(m) #0-1均匀概率分布生成方向
        #新的布谷鸟在原来的基础上加上步长和方向
        new_nests[each_nest] += step_size * step_direction  
        # apply boundary condtions
        new_nests[each_nest][new_nests[each_nest] < lower_boundary] = lower_boundary[new_nests[each_nest] < lower_boundary]
        new_nests[each_nest][new_nests[each_nest] > upper_boundary] = upper_boundary[new_nests[each_nest] > upper_boundary]

    new_fitness,new_losses,new_nests = calc_fitness(new_nests)
    #适应度更好的才更新过去
    
    nests[new_losses < lossness] = new_nests[new_losses < lossness] 
    return nests

In [25]:
def abandon_nests(nests, lower_boundary, upper_boundary, pa):

    lower_boundary = np.array(lower_boundary)
    upper_boundary = np.array(upper_boundary)
    n, m = nests.shape
    for each_nest in range(n):  
        #pa概率抛弃，抛弃后重新生成
        if (np.random.rand() < pa):
            #局部随机行走生成步长
            #随机两个种群相差*一个0-1的随机数
            step_size = np.random.rand() * (nests[np.random.randint(0, n)] - nests[np.random.randint(0, n)])
            nests[each_nest] += step_size
            # apply boundary condtions
            nests[each_nest][nests[each_nest] < lower_boundary] = lower_boundary[nests[each_nest] < lower_boundary]
            nests[each_nest][nests[each_nest] > upper_boundary] = upper_boundary[nests[each_nest] > upper_boundary]
    
    return nests

In [26]:
def levy_flight(n, m, beta):

    sigma_u = (sc_special.gamma(1+beta)*np.sin(np.pi*beta/2)/(sc_special.gamma((1+beta)/2)*beta*(2**((beta-1)/2))))**(1/beta)
    sigma_v = 1

    u =  np.random.normal(0, sigma_u, (n, m))
    v = np.random.normal(0, sigma_v, (n, m))

    steps = u/((np.abs(v))**(1/beta))

    return steps

In [27]:
def calc_fitness(nests):
    
    n, m = nests.shape
    fitness = np.empty(n)
    lossness = np.empty(n)
    new_nests = nests
    for Sig_nest in range(n):
        chrom = nests[Sig_nest]
        #分解模型参数
        w1 = chrom[:inputnum*hiddennum]
        w1 = w1.reshape(inputnum,hiddennum)
        b1 = chrom[inputnum*hiddennum:inputnum*hiddennum+hiddennum]
        w2 = chrom[inputnum*hiddennum+hiddennum:inputnum*hiddennum+hiddennum+hiddennum*outputnum]
        w2 = w2.reshape(hiddennum,outputnum)
        b2 = chrom[inputnum*hiddennum+hiddennum+hiddennum*outputnum:]

        WB_l1 = (w1,b1)
        WB_l2 = (w2,b2)
        #创建模型，并赋予参数
        model = keras.Sequential([
            keras.layers.Dense(hiddennum,activation='relu',name='l1'),
            keras.layers.Dense(outputnum,name='l2')
        ])
        model.build(input_shape=[None,inputnum])
        
        #model.summary()
        model.compile(optimizer=optimizers.Adam(lr=0.01),
                    loss='mse',
                    metrics=['accuracy'])
        
        layer1 = model.get_layer('l1')
        layer2 = model.get_layer('l2')
        layer1.set_weights(WB_l1)
        layer2.set_weights(WB_l2)
        
        #训练模型
        #model.fit(db,epochs=1,validation_data=ds_val,validation_freq=1)
        model.fit(db,epochs=2,validation_data=ds_val,validation_freq=1)
        loss,acc = model.evaluate(db)  #这里是用验证集评估，其实应该用测试集吧 evaluate产生两个结果，前面是损失，后面是准确率
        lossness[Sig_nest] = loss
        fitness[Sig_nest] = acc  #将模型评估正确率作为适应度返回
        
        (k1,y1) = layer1.get_weights()  #获取训练后的神经网络权值，并赋值给c
        (k2,y2) = layer2.get_weights()
        c=k1.reshape(1,-1).tolist()[0] + y1.reshape(1,-1).tolist()[0] + k2.reshape(1,-1).tolist()[0] + y2.reshape(1,-1).tolist()[0]
        new_nests[Sig_nest] = c
        
        
#         if(acc>bestfitness): #在计算适应度的过程当中，发现有更好的适应度就把参数保存
#             #model.save_weights('my_model_fun.h5')
#             bestfitness = acc
    return fitness,lossness,new_nests

In [28]:
# if __name__=='__main__':
low = -3*np.ones(numsum)
upp = 3*np.ones(numsum)
i = 0
j = 0
best_nest = [] #保存每次CS最佳loss对应的nest，用于赋值给不同神经网络构成多个弱分类器
best_loss = []
best_fitness = []
alpha = [0.11,0.33,0.55,0.77,0.93]  #alpha不取0.5,0<alpha<1
xn = [0.92,0.80,0.60,0.29,0.11]  #xn值不能和alpha值相同，否则将演化为周期系统，就不是混沌系统了
for i in range(M):
    print('第 %d 个CS算法开始'%(i+1))
    nest,loss,fitness = cuckoo_search(20,numsum, low,upp, step_size = 0.4,alpha=alpha[i],xn=xn[i])
    best_nest.append(nest)
    best_loss.append(loss)
    best_fitness.append(fitness)
for j in range(M):    
    print('第 %d 个CS最优loss为:%.5f!'%(j+1,best_loss[j]))



第 1 个CS算法开始
Epoch 1/2
26/26 [==============================] - 1s 9ms/step - loss: 9.2556 - accuracy: 0.4902 - val_loss: 8.4171 - val_accuracy: 0.5039
Epoch 2/2
26/26 [==============================] - 0s 800us/step - loss: 2.9195 - accuracy: 0.4683
Epoch 1/2
26/26 [==============================] - 1s 16ms/step - loss: 93.5619 - accuracy: 0.3350 - val_loss: 58.6187 - val_accuracy: 0.3488
Epoch 2/2
26/26 [==============================] - 0s 800us/step - loss: 27.0332 - accuracy: 0.5488
Epoch 1/2
26/26 [==============================] - 1s 10ms/step - loss: 32.9974 - accuracy: 0.5918 - val_loss: 28.0258 - val_accuracy: 0.5504
Epoch 2/2
26/26 [==============================] - 0s 1ms/step - loss: 9.2963 - accuracy: 0.5293
Epoch 1/2
26/26 [==============================] - 1s 10ms/step - loss: 16.6940 - accuracy: 0.7034 - val_loss: 14.7993 - val_accuracy: 0.6202
Epoch 2/2
26/26 [==============================] - 0s 1ms/step - loss: 3.1872 - accuracy: 0.6756
Epoch 1/2
26/26 [=============

26/26 [==============================] - 0s 800us/step - loss: 0.4900 - accuracy: 0.6537
Epoch 1/2
26/26 [==============================] - 1s 12ms/step - loss: 0.6692 - accuracy: 0.3920 - val_loss: 0.4536 - val_accuracy: 0.4341
Epoch 2/2
26/26 [==============================] - 0s 800us/step - loss: 0.1934 - accuracy: 0.6537
Epoch 1/2
26/26 [==============================] - 0s 8ms/step - loss: 3.5167 - accuracy: 0.4137 - val_loss: 0.9428 - val_accuracy: 0.5039
Epoch 2/2
26/26 [==============================] - 0s 800us/step - loss: 0.3390 - accuracy: 0.6146
Epoch 1/2
26/26 [==============================] - 0s 8ms/step - loss: 2.2401 - accuracy: 0.4279 - val_loss: 1.0127 - val_accuracy: 0.5039
Epoch 2/2
26/26 [==============================] - 0s 800us/step - loss: 0.6380 - accuracy: 0.6439
Epoch 1/2
26/26 [==============================] - 1s 8ms/step - loss: 6.3169 - accuracy: 0.2339 - val_loss: 2.4359 - val_accuracy: 0.3333
Epoch 2/2
26/26 [==============================] - 0s 800

26/26 [==============================] - 0s 800us/step - loss: 1.8761 - accuracy: 0.3854
Epoch 1/2
26/26 [==============================] - 0s 8ms/step - loss: 9.9304 - accuracy: 0.4192 - val_loss: 1.5285 - val_accuracy: 0.5194
Epoch 2/2
26/26 [==============================] - 0s 800us/step - loss: 0.3882 - accuracy: 0.7878
Epoch 1/2
26/26 [==============================] - 0s 8ms/step - loss: 0.7140 - accuracy: 0.5235 - val_loss: 0.9065 - val_accuracy: 0.4651
Epoch 2/2
26/26 [==============================] - 0s 799us/step - loss: 0.3166 - accuracy: 0.5829
Epoch 1/2
26/26 [==============================] - 0s 8ms/step - loss: 0.2645 - accuracy: 0.6517 - val_loss: 0.3110 - val_accuracy: 0.6512
Epoch 2/2
26/26 [==============================] - 0s 800us/step - loss: 0.1405 - accuracy: 0.6927
Epoch 1/2
26/26 [==============================] - 0s 7ms/step - loss: 1.5978 - accuracy: 0.3715 - val_loss: 2.1097 - val_accuracy: 0.2481
Epoch 2/2
26/26 [==============================] - 0s 800u

Epoch 2/2
26/26 [==============================] - 0s 1ms/step - loss: 0.3186 - accuracy: 0.6341
Epoch 1/2
26/26 [==============================] - 1s 14ms/step - loss: 38.2548 - accuracy: 0.1686 - val_loss: 17.1418 - val_accuracy: 0.3101
Epoch 2/2
26/26 [==============================] - 0s 1ms/step - loss: 5.5854 - accuracy: 0.3220
Epoch 1/2
26/26 [==============================] - 0s 9ms/step - loss: 11.6252 - accuracy: 0.5772 - val_loss: 6.6378 - val_accuracy: 0.4574
Epoch 2/2
26/26 [==============================] - 0s 800us/step - loss: 1.9107 - accuracy: 0.5341
Epoch 1/2
26/26 [==============================] - 1s 12ms/step - loss: 1.2875 - accuracy: 0.5659 - val_loss: 0.3154 - val_accuracy: 0.6744
Epoch 2/2
26/26 [==============================] - 0s 800us/step - loss: 0.1234 - accuracy: 0.7098
Epoch 1/2
26/26 [==============================] - 0s 9ms/step - loss: 1.1980 - accuracy: 0.4968 - val_loss: 0.6756 - val_accuracy: 0.6124
Epoch 2/2
26/26 [==============================

Epoch 2/2
26/26 [==============================] - 0s 800us/step - loss: 0.9233 - accuracy: 0.3927
Epoch 1/2
26/26 [==============================] - 0s 7ms/step - loss: 0.1115 - accuracy: 0.7284 - val_loss: 0.1317 - val_accuracy: 0.6667
Epoch 2/2
26/26 [==============================] - 0s 800us/step - loss: 0.0915 - accuracy: 0.7195
Epoch 1/2
26/26 [==============================] - 1s 12ms/step - loss: 0.1738 - accuracy: 0.7019 - val_loss: 0.4291 - val_accuracy: 0.6202
Epoch 2/2
26/26 [==============================] - 0s 800us/step - loss: 0.1322 - accuracy: 0.7000
Epoch 1/2
26/26 [==============================] - 0s 8ms/step - loss: 0.1245 - accuracy: 0.7261 - val_loss: 0.1443 - val_accuracy: 0.6744
Epoch 2/2
26/26 [==============================] - 0s 800us/step - loss: 0.1016 - accuracy: 0.7195
Epoch 1/2
26/26 [==============================] - 0s 7ms/step - loss: 0.3119 - accuracy: 0.5228 - val_loss: 0.3561 - val_accuracy: 0.4806
Epoch 2/2
26/26 [==============================

...

26/26 [==============================] - 0s 800us/step - loss: 0.2680 - accuracy: 0.6610
Epoch 1/2
26/26 [==============================] - 0s 7ms/step - loss: 1.3004 - accuracy: 0.3807 - val_loss: 0.8717 - val_accuracy: 0.3101
Epoch 2/2
26/26 [==============================] - 0s 799us/step - loss: 0.3627 - accuracy: 0.3585
Epoch 1/2
26/26 [==============================] - 1s 12ms/step - loss: 25.0744 - accuracy: 0.2887 - val_loss: 6.7552 - val_accuracy: 0.2946
Epoch 2/2
26/26 [==============================] - 0s 400us/step - loss: 2.9220 - accuracy: 0.3000
Epoch 1/2
26/26 [==============================] - 0s 7ms/step - loss: 8.3367 - accuracy: 0.4093 - val_loss: 2.9223 - val_accuracy: 0.5271
Epoch 2/2
26/26 [==============================] - 0s 799us/step - loss: 1.3974 - accuracy: 0.5902
Epoch 1/2
26/26 [==============================] - 0s 7ms/step - loss: 3.2707 - accuracy: 0.3661 - val_loss: 1.9799 - val_accuracy: 0.3101
Epoch 2/2
26/26 [==============================] - 0s 79

Epoch 2/2
26/26 [==============================] - 0s 799us/step - loss: 1.4464 - accuracy: 0.4415
Epoch 1/2
26/26 [==============================] - 0s 9ms/step - loss: 3.5739 - accuracy: 0.3859 - val_loss: 1.4978 - val_accuracy: 0.5426
Epoch 2/2
26/26 [==============================] - 0s 400us/step - loss: 0.7055 - accuracy: 0.5951
Epoch 1/2
26/26 [==============================] - 0s 10ms/step - loss: 0.8156 - accuracy: 0.3350 - val_loss: 0.6015 - val_accuracy: 0.3333
Epoch 2/2
26/26 [==============================] - 0s 800us/step - loss: 0.3286 - accuracy: 0.4244
Epoch 1/2
26/26 [==============================] - 1s 10ms/step - loss: 13.1851 - accuracy: 0.4526 - val_loss: 5.5900 - val_accuracy: 0.5504
Epoch 2/2
26/26 [==============================] - 0s 800us/step - loss: 1.7132 - accuracy: 0.6537
Epoch 1/2
26/26 [==============================] - 0s 8ms/step - loss: 4.2493 - accuracy: 0.6070 - val_loss: 1.3382 - val_accuracy: 0.6434
Epoch 2/2
26/26 [============================

Epoch 2/2
26/26 [==============================] - 0s 400us/step - loss: 0.1631 - accuracy: 0.5610
Epoch 1/2
26/26 [==============================] - 0s 7ms/step - loss: 0.7049 - accuracy: 0.6165 - val_loss: 0.8577 - val_accuracy: 0.5736
Epoch 2/2
26/26 [==============================] - 0s 799us/step - loss: 0.3780 - accuracy: 0.6780
Epoch 1/2
26/26 [==============================] - 0s 8ms/step - loss: 0.2649 - accuracy: 0.7689 - val_loss: 0.3742 - val_accuracy: 0.7054
Epoch 2/2
26/26 [==============================] - 0s 800us/step - loss: 0.1872 - accuracy: 0.7634
Epoch 1/2
26/26 [==============================] - 1s 12ms/step - loss: 57.4912 - accuracy: 0.2066 - val_loss: 28.9322 - val_accuracy: 0.2016
Epoch 2/2
26/26 [==============================] - 0s 800us/step - loss: 6.1160 - accuracy: 0.2927
Epoch 1/2
26/26 [==============================] - 0s 7ms/step - loss: 1.9123 - accuracy: 0.5147 - val_loss: 1.5146 - val_accuracy: 0.4109
Epoch 2/2
26/26 [============================

26/26 [==============================] - 0s 799us/step - loss: 0.1853 - accuracy: 0.7585
Epoch 1/2
26/26 [==============================] - 1s 12ms/step - loss: 0.3377 - accuracy: 0.5823 - val_loss: 0.2213 - val_accuracy: 0.6744
Epoch 2/2
26/26 [==============================] - 0s 800us/step - loss: 0.1232 - accuracy: 0.6585

*********************************************************第 19 次迭代结束************************************************************


******************************************************第 20 代开始迭代优化************************************************************

Epoch 1/2
26/26 [==============================] - 0s 8ms/step - loss: 0.1594 - accuracy: 0.5105 - val_loss: 0.1511 - val_accuracy: 0.5039
Epoch 2/2
26/26 [==============================] - 0s 799us/step - loss: 0.1509 - accuracy: 0.5195
Epoch 1/2
26/26 [==============================] - 0s 8ms/step - loss: 1.5035 - accuracy: 0.6631 - val_loss: 0.7518 - val_accuracy: 0.6124
Epoch 2/2
26/26 [===================

26/26 [==============================] - 0s 7ms/step - loss: 0.1514 - accuracy: 0.5269 - val_loss: 0.1450 - val_accuracy: 0.5426
Epoch 2/2
26/26 [==============================] - 0s 800us/step - loss: 0.1496 - accuracy: 0.5171
Epoch 1/2
26/26 [==============================] - 1s 11ms/step - loss: 0.3030 - accuracy: 0.7262 - val_loss: 0.3249 - val_accuracy: 0.6589
Epoch 2/2
26/26 [==============================] - 0s 800us/step - loss: 0.1704 - accuracy: 0.7195
Epoch 1/2
26/26 [==============================] - 0s 8ms/step - loss: 0.1147 - accuracy: 0.7653 - val_loss: 0.1433 - val_accuracy: 0.6977
Epoch 2/2
26/26 [==============================] - 0s 800us/step - loss: 0.0946 - accuracy: 0.7537
Epoch 1/2
26/26 [==============================] - 0s 7ms/step - loss: 0.0868 - accuracy: 0.7503 - val_loss: 0.1141 - val_accuracy: 0.6822
Epoch 2/2
26/26 [==============================] - 0s 400us/step - loss: 0.0801 - accuracy: 0.7439
Epoch 1/2
26/26 [==============================] - 1s 11m

# 神经网络部分

In [29]:
#用CS优化初始化阈值
Models = []
for i in range(0,M):
    chrom = best_nest[i]
    w1 = chrom[:inputnum*hiddennum]
    w1 = w1.reshape(inputnum,hiddennum)
    b1 = chrom[inputnum*hiddennum:inputnum*hiddennum+hiddennum]
    w2 = chrom[inputnum*hiddennum+hiddennum:inputnum*hiddennum+hiddennum+hiddennum*outputnum]
    w2 = w2.reshape(hiddennum,outputnum)
    b2 = chrom[inputnum*hiddennum+hiddennum+hiddennum*outputnum:]

    WB_layer1 = (w1,b1)
    WB_layer2 = (w2,b2)

    model = keras.Sequential([
        keras.layers.Dense(hiddennum,name='layer1',activation='relu'),
        keras.layers.Dense(outputnum,name='layer2')
    ])


    model.build(input_shape=[None,inputnum])
    #model.summary()
    model.compile(optimizer=optimizers.Adam(lr=0.01),
                loss='mse',
                metrics=['accuracy'])
    #model.load_weights('my_model_fun.h5')

    layer1 = model.get_layer('layer1')
    layer2 = model.get_layer('layer2')
    layer1.set_weights(WB_layer1)
    layer2.set_weights(WB_layer2)

    #训练模型
    model.fit(db,epochs=100,validation_data=ds_val,validation_freq=5)
    Models.append(model)

Epoch 1/100
26/26 [==============================] - 0s 800us/step - loss: 0.0449 - accuracy: 0.8108
Epoch 2/100
26/26 [==============================] - 0s 800us/step - loss: 0.0424 - accuracy: 0.8106
Epoch 3/100
26/26 [==============================] - 0s 800us/step - loss: 0.0408 - accuracy: 0.8140
Epoch 4/100
26/26 [==============================] - 0s 800us/step - loss: 0.0405 - accuracy: 0.8207
Epoch 5/100
26/26 [==============================] - 0s 10ms/step - loss: 0.0401 - accuracy: 0.8186 - val_loss: 0.0620 - val_accuracy: 0.7442
Epoch 6/100
26/26 [==============================] - 0s 800us/step - loss: 0.0399 - accuracy: 0.8249
Epoch 7/100
26/26 [==============================] - 0s 400us/step - loss: 0.0396 - accuracy: 0.8364
Epoch 8/100
26/26 [==============================] - 0s 800us/step - loss: 0.0392 - accuracy: 0.8351
Epoch 9/100
26/26 [==============================] - 0s 800us/step - loss: 0.0389 - accuracy: 0.8371
Epoch 10/100
26/26 [==============================

26/26 [==============================] - 0s 800us/step - loss: 0.0604 - accuracy: 0.7943
Epoch 50/100
26/26 [==============================] - 0s 6ms/step - loss: 0.0599 - accuracy: 0.7900 - val_loss: 0.1221 - val_accuracy: 0.7442
Epoch 51/100
26/26 [==============================] - 0s 800us/step - loss: 0.0593 - accuracy: 0.7876
Epoch 52/100
26/26 [==============================] - 0s 800us/step - loss: 0.0588 - accuracy: 0.7883
Epoch 53/100
26/26 [==============================] - 0s 800us/step - loss: 0.0583 - accuracy: 0.7883
Epoch 54/100
26/26 [==============================] - 0s 800us/step - loss: 0.0578 - accuracy: 0.7907
Epoch 55/100
26/26 [==============================] - 0s 2ms/step - loss: 0.0573 - accuracy: 0.7907 - val_loss: 0.1097 - val_accuracy: 0.7674
Epoch 56/100
26/26 [==============================] - 0s 400us/step - loss: 0.0569 - accuracy: 0.7963
Epoch 57/100
26/26 [==============================] - 0s 800us/step - loss: 0.0565 - accuracy: 0.7973
Epoch 58/100
26

26/26 [==============================] - 0s 800us/step - loss: 0.0307 - accuracy: 0.8535
Epoch 98/100
26/26 [==============================] - 0s 800us/step - loss: 0.0307 - accuracy: 0.8535
Epoch 99/100
26/26 [==============================] - 0s 800us/step - loss: 0.0306 - accuracy: 0.8535
Epoch 100/100
26/26 [==============================] - 0s 2ms/step - loss: 0.0307 - accuracy: 0.8535 - val_loss: 0.0619 - val_accuracy: 0.7287
Epoch 1/100
26/26 [==============================] - 0s 799us/step - loss: 0.0987 - accuracy: 0.7348
Epoch 2/100
26/26 [==============================] - 0s 400us/step - loss: 0.0962 - accuracy: 0.7466
Epoch 3/100
26/26 [==============================] - 0s 800us/step - loss: 0.0950 - accuracy: 0.7347
Epoch 4/100
26/26 [==============================] - 0s 800us/step - loss: 0.0937 - accuracy: 0.7340
Epoch 5/100
26/26 [==============================] - 0s 6ms/step - loss: 0.0920 - accuracy: 0.7469 - val_loss: 0.1741 - val_accuracy: 0.6899
Epoch 6/100
26/26 [

26/26 [==============================] - 0s 2ms/step - loss: 0.0679 - accuracy: 0.7875 - val_loss: 0.0950 - val_accuracy: 0.7132
Epoch 46/100
26/26 [==============================] - 0s 1ms/step - loss: 0.0679 - accuracy: 0.7875
Epoch 47/100
26/26 [==============================] - 0s 800us/step - loss: 0.0679 - accuracy: 0.7892
Epoch 48/100
26/26 [==============================] - 0s 800us/step - loss: 0.0679 - accuracy: 0.7892
Epoch 49/100
26/26 [==============================] - 0s 800us/step - loss: 0.0679 - accuracy: 0.7892
Epoch 50/100
26/26 [==============================] - 0s 3ms/step - loss: 0.0679 - accuracy: 0.7869 - val_loss: 0.0944 - val_accuracy: 0.7287
Epoch 51/100
26/26 [==============================] - 0s 800us/step - loss: 0.0679 - accuracy: 0.7851
Epoch 52/100
26/26 [==============================] - 0s 800us/step - loss: 0.0679 - accuracy: 0.7851
Epoch 53/100
26/26 [==============================] - 0s 800us/step - loss: 0.0679 - accuracy: 0.7851
Epoch 54/100
26/2

In [1]:
# 评估模型,不输出预测结果
#将弱分类器按准确率从大到小排序
CSMODEL = []
acc = []
for i in range(0,M):
    loss,accuracy = Models[i].evaluate(ds_val)
    acc.append(accuracy)
    
accIdx = np.argsort(acc)  #模型准确度从大到小排序，保证强分类器至少不会弱于最好的弱分类器
accIdx = accIdx[::-1]

for i in accIdx:
    CSMODEL.append(Models[i])

for i in range(0,M):
    print('第 %d 个模型的预测结果为：\r\n'%(i+1))
    loss,accuracy = CSMODEL[i].evaluate(ds_val)
    print('test loss is : %.5f'%(loss))
    print('accuracy is : %.5f\r\n'%(accuracy))
Models = CSMODEL

NameError: name 'M' is not defined

In [31]:
acc = []
for i in range(0,M):
    loss,accuracy = Models[i].evaluate(db)
    acc.append(accuracy)
acc

26/26 [==============================] - 0s 400us/step - loss: 0.0378 - accuracy: 0.7732


[0.800000011920929,
 0.8048780560493469,
 0.7658536434173584,
 0.8097561001777649,
 0.7731707096099854]

In [32]:
#查看模型参数
# la1 = model.get_layer('layer1')
# la2 = model.get_layer('layer2')
# (k1,b1) = la1.get_weights()
# (k2,b2) = la2.get_weights()
# k1,b1,k2,b2
# a=k1.reshape(1,-1).tolist()
# b=b1.reshape(1,-1).tolist()
# c = a[0] + b[0]
# len(k1.reshape(1,-1).tolist()[0] + b1.reshape(1,-1).tolist()[0] + k2.reshape(1,-1).tolist()[0] + b2.reshape(1,-1).tolist()[0])

In [33]:
y_pre_train = []
# 初始化数据权重
w_data = np.ones(n_train) / n_train
# 初始化模型权重
w_model = np.zeros(M)
train_y = tf.argmax(Y_onehot,axis=1)  #train_y保存了原始训练集中标签经过onehot编码后的结果
k = 3 #分类数量
for i in range(M):
    
    #求第i个弱分类器训练集结果
    y_pred_train = Models[i].predict(db)  #三元概率
    y_pred_train = tf.argmax(y_pred_train,axis=1)#去最大概率的位置为预测值
    
    #求第i个弱分类器误差
    miss = [int(x) for x in (y_pred_train != train_y)]  #不相等则保存1，相等则保存0
    error =np.dot(w_data, miss) #累加识别错误的样本权重，得到分类器误差
    print('第 %d 个模型的误差是：%.2f'%(i,error))
    #求第i个弱分类器权值，保存到w_model中
    #a = 1/2 * log(1-e/e) + log(k-1),当k = 2 时为二分类更新权值公式不用修改，否则为多分类，算法准确率大于1/k即可
    #该函数若准确率大于0.5（1-error）则值为正，否则为负值，越大说明模型分类越好
    w_model[i] = 0.5 * np.log((1-error)/error) + np.log(k - 1)  
    
    # 更新数据权重 
    #分类结果和真实的结果一致，那么结果是−w_model[m]，是一个负值，
    #那么exp(-w_model[m]*train_y[i]*y_pred_train[i]) 结果小于1。也就是说该数据集的样本权重降低。否则该数据样本的权重增高。
    #通过这种计算就可以让那些容易分错的样本的权重升高，容易分对的样本权重降低。继续迭代就会导致对难分的样本能分对的模型的权重上涨。
    #最终，达到一个强分类器的目的。
    #注意，这里只适合二分类【1，-1】
    #多分类公式修改 wt = wt-1 * exp(at * (y_true!=y_pred)) 
    miss1 = np.array(miss)
    miss1 = w_model[i]*miss1
    for j in range(n_train):
        w_data[j] = w_data[j] * np.exp(miss1[j])  #*train_y[i]*y_pred_train[i] #二分类时用这个
    
    #正则化数据权值
    Z = np.sum(w_data)
    for j in range(n_train):
        w_data[j] /= Z

#结果这个模块以后将得到每个模型的权值，保存在w_model中

第 0 个模型的误差是：0.20
第 1 个模型的误差是：0.40
第 2 个模型的误差是：0.68
第 3 个模型的误差是：0.59
第 4 个模型的误差是：0.78


In [34]:
#弱分类器权值归一化
w_model = np.array(w_model / np.sum(w_model))
w_model

array([0.43720875, 0.28494958, 0.10173309, 0.160334  , 0.01577458])

In [35]:
#获取弱分类器分类结果
models_pre_ds = np.array([tf.argmax(Models[m].predict(ds_val),axis = 1) for m in range(M)])
models_pre_ds

array([[2, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 0, 1, 2, 1, 1, 2, 1,
        0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1,
        1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 2,
        2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 1, 2, 0, 2,
        1, 1, 1, 2, 2, 2, 2, 2, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 2, 2, 2, 2,
        1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1],
       [2, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 0, 1, 2, 1, 1, 2, 1,
        1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1,
        2, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2,
        2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2,
        1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 0, 1, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1],
       [2, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2,
        2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1

In [36]:
 modif = models_pre_ds.copy()
modif[0][-3] = 0
modif[1][0] = 2
modif

array([[2, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 0, 1, 2, 1, 1, 2, 1,
        0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1,
        1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 2,
        2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 1, 2, 0, 2,
        1, 1, 1, 2, 2, 2, 2, 2, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 2, 2, 2, 2,
        1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 0, 2, 1],
       [2, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 0, 1, 2, 1, 1, 2, 1,
        1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1,
        2, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2,
        2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2,
        1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 0, 1, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1],
       [2, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2,
        2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1

In [37]:
# 最终的预测
result1 = w_model[0]*modif[0]

for i in range(1,M):
    result1 += w_model[i] * modif[i]
result1 = np.round(result1)
result1 = result.astype(int)
result1

NameError: name 'result' is not defined

# 强分类器测试

In [ ]:
# 最终的预测
result = w_model[0]*models_pre_ds[0]

for i in range(1,M):
    result += w_model[i] * models_pre_ds[i]
result = np.round(result)
result= result.astype(int)
result

In [ ]:
test_y = tf.argmax(TestY_onehot,axis=1)


In [ ]:
sum = 0
for i in range(len(result1)):
    if result1[i]==test_y[i]:
        sum=sum+1
sum/len(result1)

# 绘图部分

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
sns.set_style("ticks")
x = list(range(0,len(LossArr)))
fig = sns.lineplot(x,LossArr)
plt.xlabel("epoch")
plt.ylabel("fitness")
scatter_fig = fig.get_figure()
scatter_fig.savefig('./BestFit', dpi = 400)

In [ ]:
#弱分类器
error = models_pre_ds[-1] - test_y
x = list(range(0,len(error)))
fig = sns.lineplot(x,error,color="b")
plt.xlabel("Test Data Num")
plt.ylabel("error")
scatter_fig = fig.get_figure()
scatter_fig.savefig('./Error_Num', dpi = 400)



In [ ]:
#强分类器
error = result - test_y
x = list(range(0,len(error)))
fig = sns.lineplot(x,error,color="b")
plt.xlabel("Test Data Num")
plt.ylabel("error")
scatter_fig = fig.get_figure()
scatter_fig.savefig('./result', dpi = 400)



In [ ]:
len(bestloss),len(bestfit)

In [ ]:
x = list(range(0,len(bestloss)))
fig = sns.lineplot(x,bestloss,color="r")
plt.xlabel("epoch")
plt.ylabel("lossness")
scatter_fig = fig.get_figure()
scatter_fig.savefig('./BestLoss', dpi = 400)

In [ ]:
#sns主题风格  darkgrid  whitegrid  dark  white  ticks
sns.set_style("ticks")  #设置主题风格
sns.color_palette("hls",8)  #设置颜色空间种类（几种可用颜色）
data=np.random.normal(size=(20,8)) + np.arange(8) /2
sns.boxplot(data = data,palette = sns.color_palette("hls",8))

# 描述两个变量的关系 最好用散点图

In [ ]:
import pandas as pd
mean,cov = [0,1],[(1,.5),(.5,1)]
data = np.random.multivariate_normal(mean,cov,200)
df = pd.DataFrame(data,columns=["x","y"])
#绘制散点图
sns.jointplot(x="x",y="y",data = df,color="r")  #如果点很多，用颜色深度表示数量 kind="hex" ,可以单独传x y

# 移植STM32准备

In [ ]:
Testx = X / X.max().max()
Testx = np.array(Testx)

In [ ]:
aa = []
aa.append(list(Testx[0]))
aa

In [ ]:
pre_sig=model.predict(aa)
pre_sig

In [ ]:
index = np.argmax(pre_sig,axis=None)
index

In [ ]:
Y

# 模型保存

In [ ]:
model.save('Env.h5')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(load_model)
tflite_model = converter.convert()
open("level_check.tflite", "wb").write(tflite_model)

In [ ]:
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPooling2D
from tensorflow.keras import Sequential
import tensorflow.keras as keras
import numpy as np

In [ ]:


# #建立一个测试模型
model = Sequential([
    Dense(4, input_shape=(5,), name='dense_xiaoming',
          kernel_initializer='zeros',  # 全部初始化为0
          bias_initializer='ones'),  # 全部初始化为1
])



In [ ]:
x = np.ones(shape=(8, 5))  # 创建测试数据
y = model(x)
layer = model.get_layer('dense_xiaoming')  # 通过层的名字得到层
(k, b) = layer.get_weights()  # 查看层的初始化权重值和偏置项
print(k)
print(b)

In [ ]:
layer